In [32]:
from pprint import pprint
import requests
import numpy
import math
from hydradx.model.processing import save_latest_coingecko_prices, import_coingecko_prices

import time
import datetime
# now = datetime.datetime.now()
# buffer_time = 10  # in seconds
# duration_ls = [1,7,30,90,180]  # in days
# 
# recent = now - datetime.timedelta(seconds=buffer_time)
# d2_unix = str(int(time.mktime(recent.timetuple())))
# asset = 'hydradx'
# # asset = 'interlay'
# 
# responses = {}
# for duration in duration_ls:
#     past = recent - datetime.timedelta(days=duration)
#     d1_unix = str(int(time.mktime(past.timetuple())))
#     url = "https://api.coingecko.com/api/v3/coins/" + asset + "/market_chart/range?vs_currency=usd&from=" + d1_unix + "&to=" + d2_unix
#     response = requests.get(url)
#     responses[duration] = response
duration_ls = [1,7,30,90,180]  # in days
asset = 'hydradx'

download = False
if download:
    for duration in duration_ls:
        save_latest_coingecko_prices(asset, duration)
responses = {}
for duration in duration_ls:
    responses[duration] = import_coingecko_prices(asset, duration)


In [33]:
vols = {}
for duration in responses:
    response = responses[duration]
    if 'prices' not in response:
        raise
    price_pairs = response['prices']
    prices = [x[1] for x in price_pairs]
    timestamps = [x[0] for x in price_pairs]
    # list of pairs (timestamp, price)
    # timestamp is in milliseconds, unix format
    
    # for 1Y of daily data, we can calculate 30D diffs by
    delta = 1
    N = (len(prices) - 1) // delta
    price_ratios = [prices[(i+1)*delta]/prices[i*delta] for i in range(N)]
    lns = [math.log(x) for x in price_ratios]
    vols[duration] = numpy.std(lns)*math.sqrt(N)
pprint(vols)


{1: 0.03389905232385241,
 7: 0.14211249399602083,
 30: 0.4161888494015759,
 90: 0.735653699405586,
 180: 0.8347739263585945}
